In [1]:
from dotenv import load_dotenv
load_dotenv()
import textwrap

In [2]:

import time
from PyPDF2 import PdfReader
from PIL import Image

import os


# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
import openai
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import ChatGLM
from langchain.prompts import PromptTemplate


In [3]:
# default endpoint_url for a local deployed ChatGLM api server
endpoint_url = "https://f7ad-35-227-93-64.ngrok-free.app"

# direct access endpoint in a proxied environment
# os.environ['NO_PROXY'] = '127.0.0.1'

llm = ChatGLM(
    endpoint_url=endpoint_url,
    max_token=80000,
    history=[],
    top_p=0.9,
)

In [4]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text


In [5]:
text=extract_text_from_pdf("/home/nikhil/Documents/Github/Sumquery-PDF-AiSummarizer/trash/pacific-rim-2013.pdf")

In [50]:
# text="""Elon Reeve Musk (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and chief technology officer of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is the wealthiest person in the world, with an estimated net worth of US$222 billion as of December 2023, according to the Bloomberg Billionaires Index, and $244 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]

# A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania, and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University. However, Musk dropped out after two days and, with his brother Kimbal, co-founded online city guide software company Zip2. The startup was acquired by Compaq for $307 million in 1999, and, that same year Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal.

# In October 2002, eBay acquired PayPal for $1.5 billion, and that same year, with $100 million of the money he made, Musk founded SpaceX, a spaceflight services company. In 2004, he became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.). He became its chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2022, he acquired Twitter for $44 billion. He subsequently merged the company into newly created X Corp. and rebranded the service as X the following year. In March 2023, he founded xAI, an artificial intelligence company.

# Musk has expressed views that have made him a polarizing figure.[7][8][9] He has been criticized for making unscientific and misleading statements, including COVID-19 misinformation, transphobia[10][11][12] and antisemitic conspiracy theories.[7][13][14][15] His ownership of Twitter has been similarly controversial, including; laying off a large number of employees, an increase in hate speech and misinformation and disinformation on the website, as well as changes to Twitter Blue verification. In 2018, the U.S. Securities and Exchange Commission (SEC) sued him for falsely tweeting that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine.
# """

In [10]:
prompt_template = """
Given the following text, extract and list down the important scenes:
Also,please add some of descriptions
---

{text}

---

List of Scenes:
1. Extracted Scene 1
2. Extracted Scene 2
3. Extracted Scene 3
4. Extracted Scene 4
5. Extracted Scene 5

---
"""

In [17]:
text_splitter = CharacterTextSplitter(
separator="\n",
chunk_size=1000,
chunk_overlap=200,
length_function=len
)

chunks = text_splitter.split_text(text)
    
# create docs
docs = [Document(page_content=t) for t in chunks[:10]]


# show summarize doc
BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="map_reduce")
summarized_docs = chain.run(docs)


/home/nikhil/Documents/Github/Sumquery-PDF-AiSummarizer/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Token indices sequence length is longer than the specified maximum sequence length for this model (1517 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
print("Summary")
print(summarized_docs)
wrapped_text = textwrap.wrap(summarized_docs, width=100)

# Print the wrapped text
for line in wrapped_text:
    print(line)

In [23]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

In [28]:
# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [29]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [30]:
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [31]:
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)


In [32]:
print(map_reduce_chain.run(split_docs))

The main themes of the Pacific Rim Lexicon are:

1. The Jaeger universe and its various factions or factions.
2. The use of advanced technology, including but not limited to:
	* Anteverse
	* BTB
	* conn-pod
	* COSDEC
	* drift
	* drivesuit
	* ghost-drift
3. The concept of the Pons, a device for linking Jaeger crew members together.
4. The term "bruiser," which refers to a Jaeger gunner.
5. The term "BTB," which refers to brain-to-brain interface.
6. The term "conn-pod," which refers to the Jaeger's detachable cockpit module.
7. The term "COSDEC," which refers to the Pacific Rim Combined Special Defense Corp.
8. The concept of drift, which refers to a shared vision among Jaeger crew members.
9. The use of the "drivesuit," which is the Jaeger/pilot interface suit designed to monitor vital signs and translate nerve signals to piloting input.
10. The term "ghost-drift," which refers to a kind of quasi-telepathic vision shared between parties with a pre-existing Pons connection.

The main th